# My CycleGAN is somewhat of a painter itself

This notebook is part of the Machine Learning in Practice Course 2021 at Radboud University. With it, we are participating in the "I am somewhat of a painter myself" challenge.

**Objective of the challenge**: Build a GAN that generates 7,000 to 10,000 Monet-style images

## Outline of this notebook

0. <a href='#imports'>Imports</a>
1. <a href='#eda'>Exploratory Data Analysis (EDA) (short version)</a>
2. <a href='#data_loading'>Data Loading</a>
3. <a href='#data_augmentation'>Data Augmentation</a>
4. <a href='#model'>The model</a>
    1. <a href='#generator'>Generator</a>
    2. <a href='#discriminator'>Discriminator</a>
    3. <a href='#cycle'>CycleGan</a>
    4. <a href='#losses'>Losses</a>
5. <a href='#training'>Model Training</a>
6. <a href='#results'>Results</a>
7. <a href='#eda2'>Exhaustive EDA</a>

<a id='imports'></a>
## Imports

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

from tensorflow.keras.layers import LeakyReLU, BatchNormalization, Conv2D, Add, Layer, Conv2DTranspose, Activation, ZeroPadding2D, Input
from tensorflow_addons.layers import InstanceNormalization
from keras.models import Model
from keras.activations import *
from keras.layers import *

from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt
import numpy as np

import os, random, json, PIL, shutil, re, imageio, glob

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

AUTOTUNE = tf.data.experimental.AUTOTUNE
    
print(tf.__version__)

<a id='eda'></a>
## Exploratory Data Analysis


## Visualization of a few images

Let's have a first look at the data we got. All the images for the challenge are of the size 256x256 and sorted by type already

In [ ]:
BASE_PATH = '../input/gan-getting-started'
MONET_PATH = os.path.join(BASE_PATH, 'monet_jpg')
PHOTO_PATH = os.path.join(BASE_PATH, 'photo_jpg')

import cv2
import math
import random


def load_images(paths):
    images = []
    for img in paths:
        try:
            img = cv2.imread(img)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        except:
            print("Could not load {}".format(img))
        images.append(img)
    return images
        
        
def show_folder_info(path):
    d_image_sizes = {}
    for image_name in os.listdir(path):
        image = cv2.imread(os.path.join(path, image_name))
        d_image_sizes[image.shape] = d_image_sizes.get(image.shape, 0) + 1
        
    for size, count in d_image_sizes.items():
        print(f'shape: {size}\tcount: {count}')
        
def visualize_images(images, title=None):
    plt.figure(figsize=(16,16))
    
    w = int(len(images) ** .5)
    h = math.ceil(len(images) / w)
    
    if title:
        plt.suptitle(title)
        
    for idx, image in enumerate(images):
        
        plt.subplot(h, w, idx + 1)
        plt.imshow(image)
        plt.axis('off')
    
    
    
    plt.show()
        

In [ ]:
MONET_IMAGES = [os.path.join(MONET_PATH, file) for file in os.listdir(MONET_PATH)]
monet_images = load_images(MONET_IMAGES)

PHOTO_IMAGES = [os.path.join(PHOTO_PATH, file) for file in os.listdir(PHOTO_PATH)]
photo_images = load_images(PHOTO_IMAGES)

### Monet Style Images

In [ ]:
visualize_images(random.sample(monet_images,15), "Samples of the Monet Dataset")

### Photos

In [ ]:
visualize_images(random.sample(photo_images,15), "Samples of the Photo Dataset")

*Visual Inspection*

Comparing the Monet images and the photos we can obviously see which are paintings and which are photos. The photos have way smoother transitions, the monet images "jump" in colours. Monet was a impressionist painter, an art style which is characterized by a small but still visible brush strokes. 
So our final images should also have this characteristic but showing the content of the content images.  

Further, the colours of photos are more natural than the colours of the mone style images. This is not too surprising, but we want to keep this in mind when judging the final output of our GAN

### Data Stats:

In [ ]:
print("Monet Picture Overview")
show_folder_info(MONET_PATH)

print("Content Picture Overview")
show_folder_info(PHOTO_PATH)

<a id='data_loading'></a>
## Data Loading

In [ ]:
GCS_PATH_FULL_DATA = KaggleDatasets().get_gcs_path("gan-getting-started")
GCS_PATH_CLEAN_DATA = KaggleDatasets().get_gcs_path("cleaned-monet-data")

In [ ]:
excluded =  ['05144e306f.jpg',
             '9d9a4fccfb.jpg',
             '3283442e33.jpg',
             'b5c2fe7c4c.jpg',
             '8ee2933868.jpg',
             'b1ea5d5a7d.jpg',
             'cdddf326e3.jpg',
             'cb50326950.jpg',
             '23d6aeb485.jpg',
             '47a0548067.jpg',
             '9963d64ebf.jpg',
             '16dabe418c.jpg',
             'c78b4fa3a9.jpg',
             '23b07c3769.jpg',
             '2e0d0e6e19.jpg',
             '6a03aea8be.jpg']

excluded_images = load_images([f"{MONET_PATH}/{img}" for img in excluded])

We load a cleaned version of the monet dataset. The original had two round images and some really dark and not quite "Monet-like" images. The following images were excluded:


In [ ]:
visualize_images(excluded_images, "Images excluded from dataset")

In [ ]:
MONET_FILENAMES = tf.io.gfile.glob(str(GCS_PATH_CLEAN_DATA + '/*.tfrec'))
PHOTO_FILENAMES = tf.io.gfile.glob(str(GCS_PATH_FULL_DATA + '/photo_tfrec/*.tfrec'))

The images for this challenges are already cleaned. All are of size 256x256 and have three channels (RGB)

In [ ]:
IMAGE_SIZE = [256, 256]
HEIGHT_IMG = 256
WIDTH_IMG = 256
N_CHANNELS = 3 

Below we define two functions. `read_tfrecord(example)` retrieves the images data from the tf record, `decode_image` transforms it into a `tf.image`

In [ ]:
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = (tf.cast(image, tf.float32) / 127.5) - 1
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

def read_tfrecord(example):
    tfrecord_format = {
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    return image

Lastly, we create a function that loads the dataset and combines the upper functions

In [ ]:
def load_dataset(filenames, labeled=True, ordered=False):
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTOTUNE)
    return dataset

<a id='data_augmentation'></a>
## Data Augmentation

Data augmentation can help to improve our model. However, we need to make sure to not change the style too much, as otherwise we might decrease the performance of our model. We therefore decided to only apply augmentation strategies which will not change the image style too much. This includes:

- Mirroring
- Rotation
- Changes in Saturation
- Cropping


In [ ]:
def data_augment_crop(image):
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    
    if p_crop > .5:
        height_crop = 275
        width_crop = 275
        image = tf.image.resize(image, [height_crop, width_crop])
        image = tf.image.random_crop(image, size=[HEIGHT_IMG, WIDTH_IMG, N_CHANNELS])
        if p_crop > .9:
            height_crop = 300
            width_crop = 300
            image = tf.image.resize(image, [height_crop, width_crop])
            image = tf.image.random_crop(image, size=[HEIGHT_IMG, WIDTH_IMG, N_CHANNELS])
            
    return image

def data_augment_mirror(image):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)

    if p_spatial > .5:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        if p_spatial > .9:
            image = tf.image.transpose(image)
            
    return image

def data_augment_rotate(image):
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)

    if p_rotate > .9:
        image = tf.image.rot90(image, k=3) # rotate 270º
    elif p_rotate > .7:
        image = tf.image.rot90(image, k=2) # rotate 180º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=1) # rotate 90º
            
    return image

def data_augment_color(image):
    p_color = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    x = tf.image.random_saturation(image, 0.5, 1.5)
    
    if p_color > .5:
        x = tf.image.random_brightness(x, 0.05)
    else:
        x = tf.image.random_contrast(x, 0.5, 1.5)

    return image  

Lastly, we need a function that combines all our data loading functions

In [ ]:
def process_dataset(filenames, augment = False, repeat = True, shuffle = False, label = True, batch_size = 1):
    ds = load_dataset(filenames, labeled = label)
    
    if augment:
        ds = ds.map(data_augment_crop, num_parallel_calls=AUTOTUNE)
        ds = ds.concatenate(load_dataset(filenames))
        ds = ds.map(data_augment_mirror, num_parallel_calls=AUTOTUNE)
        ds = ds.concatenate(load_dataset(filenames))
        ds = ds.map(data_augment_color, num_parallel_calls=AUTOTUNE)
        ds = ds.concatenate(load_dataset(filenames))
        ds = ds.map(data_augment_rotate, num_parallel_calls=AUTOTUNE)
        ds = ds.concatenate(load_dataset(filenames))
    if repeat:
        ds = ds.repeat(count=1)
    
    if shuffle:
        ds = ds.shuffle(2021)
    
    ds = ds.batch(batch_size,drop_remainder=True)
    ds = ds.cache()
    ds = ds.prefetch(AUTOTUNE)
    
    return ds

In [ ]:
batch_s = 32
monet_ds = process_dataset(MONET_FILENAMES, augment = True, shuffle = True, batch_size = batch_s)
photo_ds = process_dataset(PHOTO_FILENAMES, augment = False, shuffle = True, batch_size = batch_s)

<a id='model'></a>
## The Model

<a id='generator'></a>
### Generator

<img src= "https://imgur.com/AhIJfHj.png" alt ="Titanic" style='width: 500px;'>

In [ ]:
def residual_block(x):
    layer_block = x
    x = Conv2D(128, (3, 3), strides=1, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), strides=1, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    m = layers.add([x, layer_block])
    return m

In [ ]:
def effnet_block(x_in, ch_in, ch_out):
    x = Conv2D(ch_in, kernel_size=(1, 1), padding='same', use_bias=False)(x_in) #x_in
    x = get_post(x)

    x = DepthwiseConv2D(kernel_size=(1, 3), padding='same', use_bias=False)(x)
    x = get_post(x)
    x = MaxPool2D(pool_size=(2, 1), strides=(2, 1))(x)
    
    x = DepthwiseConv2D(kernel_size=(3, 1), padding='same', use_bias=False)(x)
    x = get_post(x)

    x = Conv2D(ch_out, kernel_size=(2, 1), strides=(1, 2), padding='same', use_bias=False)(x)
    x = get_post(x)

    return x

def get_post(x_in):
    x = LeakyReLU()(x_in)
    x = BatchNormalization()(x)
    return x

In [ ]:
def create_generator():
    input_layer = layers.Input(shape=(256, 256, 3), dtype='float32')
    
    ef1 = effnet_block(input_layer, 32, 64)
    ef2 = effnet_block(ef1, 64, 128)
    
    r1 = residual_block(ef2)
    r2 = residual_block(r1)
    r3 = residual_block(r2)
    r4 = residual_block(r3)
    r5 = residual_block(r4)

    d1 = Conv2DTranspose(64, (3, 3), strides=2, padding='same', use_bias=False)(r5)
    d1 = BatchNormalization()(d1)
    d1 = LeakyReLU(alpha=0.05)(d1)

    d2 = Conv2DTranspose(32, (3, 3), strides=2, padding='same', use_bias=False)(d1)
    d2 = BatchNormalization()(d2)
    d2 = LeakyReLU(alpha=0.05)(d2)

    c1 = Conv2D(3, (9, 9), strides=1, padding='same', use_bias=False)(d2)
    c1 = BatchNormalization()(c1)
    
    output_layer = Activation('tanh')(c1)

    model = Model([input_layer], output_layer)
    return model

![](http://)<a id='discriminator'></a>
<div style="display: flex; flex-direction: column; align-items: center">
    <div>
    <h3> Discriminator </h3>
        <p>Our discriminator follows the following architecture:</p></div>
<img src= "https://imgur.com/eJliJxm.png" alt ="Our Discriminator" style='width: 300px;'></div>

In [ ]:
def create_discriminator():
    input_layer = layers.Input(shape=[256, 256, 3])
    input_noise = layers.GaussianNoise(0.2)(input_layer) #0.1 #input_layer

    d1 = layers.Conv2D(64, 4, strides=2, padding='same', use_bias=False)(input_noise)
    d1 = layers.LeakyReLU(alpha=0.05)(d1)
    d2 = layers.Conv2D(128, 4, strides=2, padding='same', use_bias=False)(d1)
    d2 = layers.LeakyReLU(alpha=0.05)(d2)
    d3 = layers.Conv2D(256, 4, strides=2, padding='same', use_bias=False)(d2)
    d3 = layers.LeakyReLU(alpha=0.05)(d3)
    z1 = layers.ZeroPadding2D()(d3)
    
    c1 = layers.Conv2D(512, 4, strides=1, use_bias=False)(z1)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.LeakyReLU()(c1)
    z2 = layers.ZeroPadding2D()(c1)

    out_layer = layers.Conv2D(1, 4, strides=1, use_bias=False)(z2)

    return tf.keras.Model(inputs=input_layer, outputs=out_layer)

## Inception Model

To encourage our model to adapt the style of monet images, we extract features for both generated and original Monet images from middle layers of the Inception V3 model. 
At a later point we add an additional term to our losses that requires the model to minimize the distance between the layer output of the real monet and the current training sample. 

This way, we hope to capture the pure style features (what may be recognized as brush strokes, colours etc. for humans) and add an additional constraint to the generation process.

In [ ]:
def create_activation_model():
    
    inception_model = tf.keras.applications.InceptionV3(input_shape=(256,256,3),pooling="avg",include_top=False, weights = "imagenet", )
    activations = []
    
    # choose a middle layer of the activation layers. Choose on with 160 channels for performance reasons
    for layer in inception_model.layers:
        if "activation" in layer.name and layer.output.shape[3] == 160:
            activations.append(layer)
            
    # choose the middle of those activations
    layer = inception_model.get_layer(activations[int(len(activations)/2)].name)
    
    print(f'Selected layer {layer} for the activation model')
    
    return tf.keras.models.Model(inputs=inception_model.input, outputs=[layer.output])

<a id='cycle'></a>
### CycleGan Architecture

First, we create the instances of our generator and discriminator

In [ ]:
with strategy.scope():
    monet_generator = create_generator() # transforms photos to Monet-esque paintings
    photo_generator = create_generator() # transforms Monet paintings to be more like photos

    monet_discriminator = create_discriminator() # differentiates real Monet paintings and generated Monet paintings
    photo_discriminator = create_discriminator() # differentiates real photos and generated photos
    
    activation_model = create_activation_model()

And then combine everything in the cycleGAN architecture

In [ ]:
class CycleGan(keras.Model):
    def __init__(
        self,
        monet_generator,
        photo_generator,
        monet_discriminator,
        photo_discriminator,
        activation_model,
        mu,
        lambda_cycle=10, #25
    ):
        super(CycleGan, self).__init__()
        self.m_gen = monet_generator
        self.p_gen = photo_generator
        self.m_disc = monet_discriminator
        self.p_disc = photo_discriminator
        
        # add the inception model to the mix
        self.inception = activation_model
        self.inception.trainable = False
        # the average  monet activation map
        self.mu_monet = mu
        
        self.lambda_cycle = lambda_cycle
        
    def compile(
        self,
        m_gen_optimizer,
        p_gen_optimizer,
        m_disc_optimizer,
        p_disc_optimizer,
        gen_loss_fn,
        gen_loss_photo,
        disc_loss_fn,
        cycle_loss_fn,
        identity_loss_fn
    ):
        super(CycleGan, self).compile()
        self.m_gen_optimizer = m_gen_optimizer
        self.p_gen_optimizer = p_gen_optimizer
        self.m_disc_optimizer = m_disc_optimizer
        self.p_disc_optimizer = p_disc_optimizer
        self.gen_loss_fn = gen_loss_fn
        self.gen_loss_photo = gen_loss_photo
        self.disc_loss_fn = disc_loss_fn
        self.cycle_loss_fn = cycle_loss_fn
        self.identity_loss_fn = identity_loss_fn
        
    def train_step(self, batch_data):
        real_monet, real_photo = batch_data
        
        with tf.GradientTape(persistent=True) as tape:
            # photo to monet back to photo
            fake_monet = self.m_gen(real_photo, training=True)
            cycled_photo = self.p_gen(fake_monet, training=True)

            # monet to photo back to monet
            fake_photo = self.p_gen(real_monet, training=True)
            cycled_monet = self.m_gen(fake_photo, training=True)

            # generating itself
            same_monet = self.m_gen(real_monet, training=True)
            same_photo = self.p_gen(real_photo, training=True)

            # discriminator used to check, inputing real images
            disc_real_monet = self.m_disc(real_monet, training=True)
            disc_real_photo = self.p_disc(real_photo, training=True)

            # discriminator used to check, inputing fake images
            disc_fake_monet = self.m_disc(fake_monet, training=True)
            disc_fake_photo = self.p_disc(fake_photo, training=True)

            #inception model output
            inception_fake_monet = self.inception(fake_monet)
            
            # evaluates generator loss
            monet_gen_loss = self.gen_loss_fn(disc_fake_monet, inception_fake_monet, self.mu_monet, self.lambda_cycle)
            photo_gen_loss = self.gen_loss_photo(disc_fake_photo)
            
            # evaluates total cycle consistency loss
            total_cycle_loss = self.cycle_loss_fn(real_monet, cycled_monet, self.lambda_cycle) + self.cycle_loss_fn(real_photo, cycled_photo, self.lambda_cycle)

            # evaluates total generator loss
            total_monet_gen_loss = monet_gen_loss + total_cycle_loss + self.identity_loss_fn(real_monet, same_monet, self.lambda_cycle)
            total_photo_gen_loss = photo_gen_loss + total_cycle_loss + self.identity_loss_fn(real_photo, same_photo, self.lambda_cycle)

            # evaluates discriminator loss
            monet_disc_loss = self.disc_loss_fn(disc_real_monet, disc_fake_monet)
            photo_disc_loss = self.disc_loss_fn(disc_real_photo, disc_fake_photo)

        # Calculate the gradients for generator and discriminator
        monet_generator_gradients = tape.gradient(total_monet_gen_loss,
                                                  self.m_gen.trainable_variables)
        photo_generator_gradients = tape.gradient(total_photo_gen_loss,
                                                  self.p_gen.trainable_variables)

        monet_discriminator_gradients = tape.gradient(monet_disc_loss,
                                                      self.m_disc.trainable_variables)
        photo_discriminator_gradients = tape.gradient(photo_disc_loss,
                                                      self.p_disc.trainable_variables)

        # Apply the gradients to the optimizer
        self.m_gen_optimizer.apply_gradients(zip(monet_generator_gradients,
                                                 self.m_gen.trainable_variables))

        self.p_gen_optimizer.apply_gradients(zip(photo_generator_gradients,
                                                 self.p_gen.trainable_variables))

        self.m_disc_optimizer.apply_gradients(zip(monet_discriminator_gradients,
                                                  self.m_disc.trainable_variables))

        self.p_disc_optimizer.apply_gradients(zip(photo_discriminator_gradients,
                                                  self.p_disc.trainable_variables))
        
        return {
            "monet_gen_loss": total_monet_gen_loss,
            "photo_gen_loss": total_photo_gen_loss,
            "monet_disc_loss": monet_disc_loss,
            "photo_disc_loss": photo_disc_loss
        }

<a id='losses'></a>
### Losses

In [ ]:
with strategy.scope():
    def discriminator_loss(real, generated):
        real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(tf.ones_like(real), real)

        generated_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(tf.zeros_like(generated), generated)

        total_disc_loss = real_loss + generated_loss

        return total_disc_loss * 0.5

In [ ]:
with strategy.scope():
    
    # for the generated monet, combine BCE with our custom loss based on the activation layers
    def generator_loss(generated, inception_fake, mu, LAMBDA):
        loss = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(tf.ones_like(generated), generated)
        
        return loss + LAMBDA * tf.reduce_mean(tf.abs(mu - inception_fake))
    
    # fpr the transformation back keep the BCE
    def generator_loss_photo(generated):
        return tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(tf.ones_like(generated), generated)
        

In [ ]:
with strategy.scope():
    def calc_cycle_loss(real_image, cycled_image, LAMBDA):
        loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))

        return LAMBDA * loss1

In [ ]:
with strategy.scope():
    def identity_loss(real_image, same_image, LAMBDA):
        loss = tf.reduce_mean(tf.abs(real_image - same_image))
        return LAMBDA * 0.5 * loss

<a id='training'></a>
## Training

In [ ]:
with strategy.scope():
    monet_generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    photo_generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

    monet_discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    photo_discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

Compute the average activation for the monet images. The inception model is just used for feature extraction which means we have to compute the activation map for the original model only once

In [ ]:
monet_ds_single_batch = process_dataset(MONET_FILENAMES, augment = False, shuffle = True, batch_size = 1)

activations = []

for batch, images in enumerate(monet_ds_single_batch):
    pred = activation_model(images)
    activations.append(pred)

# average over all images
mu_monet = np.mean(activations, axis=0)
       

In [ ]:
with strategy.scope():
    cycle_gan_model = CycleGan(
        monet_generator, photo_generator, monet_discriminator, photo_discriminator, activation_model, mu_monet
    )

    cycle_gan_model.compile(
        m_gen_optimizer = monet_generator_optimizer,
        p_gen_optimizer = photo_generator_optimizer,
        m_disc_optimizer = monet_discriminator_optimizer,
        p_disc_optimizer = photo_discriminator_optimizer,
        gen_loss_fn = generator_loss,
        gen_loss_photo = generator_loss_photo,
        disc_loss_fn = discriminator_loss,
        cycle_loss_fn = calc_cycle_loss,
        identity_loss_fn = identity_loss
    )

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
cycle_gan_model.fit(
    tf.data.Dataset.zip((monet_ds, photo_ds)),
    epochs=75,
    callbacks=[callback]
)

<a id='results'></a>
## Results

Visualization of the results

In [ ]:
_, ax = plt.subplots(5, 2, figsize=(50,50))
ds_predict = load_dataset(PHOTO_FILENAMES).batch(1)
for i, img in enumerate(ds_predict.take(5)):
    prediction = monet_generator(img, training=False)[0].numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    img = (img[0] * 127.5 + 127.5).numpy().astype(np.uint8)

    ax[i, 0].imshow(img)
    ax[i, 1].imshow(prediction)
    ax[i, 0].set_title("Input Photo")
    ax[i, 1].set_title("Monet-esque (generated)")
    ax[i, 0].axis("off")
    ax[i, 1].axis("off")
plt.show()

In [ ]:
import PIL
! mkdir ../images

In [ ]:
fast_photo_ds = load_dataset(PHOTO_FILENAMES).batch(32*strategy.num_replicas_in_sync).prefetch(32)

In [ ]:
%%time
i = 1
for img in fast_photo_ds:
    prediction = monet_generator(img, training=False).numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    for pred in prediction:
        im = PIL.Image.fromarray(pred)
        im.save("../images/" + str(i) + ".jpg")
        i += 1

In [ ]:
print(f"Generated {i} images for submission")

In [ ]:
import shutil
shutil.make_archive("/kaggle/working/images", 'zip', "/kaggle/images")

<a id='eda2'></a>
## Exhaustive EDA

### Histogram analysis

OpenCV uses the BGR channel order, thus the first histogram of an image is its blue value histogram, the second gree and the third red

In [ ]:
monet_histograms = [[cv2.calcHist([image],[i],None,[256],[0,256]) for i in range(3)] for image in monet_images]
photo_histograms = [[cv2.calcHist([image],[i],None,[256],[0,256]) for i in range(3)] for image in photo_images]

In [ ]:
def plot_histograms(img, title):
    plt.figure(figsize=(16,16))
    
    plt.title(title)
    color = ('b','g','r')
    
    
    w = 4
    h = int(len(img)/2)
    
    idxs = [i for i in range(len(img)*2)[::2]]
    
    for idx, image in zip(idxs,img):
        
        plt.subplot(h, w, idx + 1)
        plt.imshow(image)
        plt.axis('off')
        
        plt.subplot(h, w, idx + 2)
        for i,col in enumerate(color):   
            histr = cv2.calcHist([image],[i],None,[256],[0,256])
            plt.plot(histr,color = col)
            plt.xlim([0,256])

In [ ]:
plot_histograms(random.sample(monet_images,8), "Histogram for Monet Image")

In [ ]:
plot_histograms(random.sample(photo_images,8), "Histogram for Photo")

The histograms indicate that 

(1) The monet images seem to use on average more colours for every channel. We come to this conclusion, since the mean pixel intensity for the Monet images is higher than for the photos. However, given the amount of photo files, there are several outliers in both directions. 

(2) The range of intensity values used for the Monet images is smaller than for the photos. I.e. the Monet images have more "unused" intensity values. This may be explained by the fact that photo transitions are way more smooth than the transitions of colours in expressionistic paintings. 

However, these observations are not too surprising considering that we have paintings and images.

### Analysis of the image intensity value distributions

In [ ]:
color = ('b','g','r')
monet_means = {c: [np.mean(x[:,:,i]) for x in monet_images] for i,c in enumerate(color)}
photo_means = {c: [np.mean(x[:,:,i]) for x in photo_images] for i,c in enumerate(color)}

monet_std = {c: [np.std(x[:,:,i]) for x in monet_images] for i,c in enumerate(color)}
photo_std = {c: [np.std(x[:,:,i]) for x in photo_images] for i,c in enumerate(color)}

In [ ]:
def boxplots_for_comparison(monet, photo, title):
    for i,c in enumerate(color):
        fig, ax = plt.subplots()
        ax.set_title('{} Channel: {}'.format(c.upper(), title))
        ax.boxplot([monet[c], photo[c]])
        ax.set_xticklabels(["Monet","Photo"])
        fig.show()

In [ ]:
boxplots_for_comparison(monet_means, photo_means, "Pixel Value Mean comparison - Monet and Photo")

In [ ]:
boxplots_for_comparison(monet_std, photo_std, "Pixel Value standard deviation comparison - Monet and Photo")

#### "Frequency" of intensity values

The boxplots only tell us the mean and the standard deviation of the intensity values. Now it would be interesting to see if both Monet and the photos make use of the full range of intensity values. That is, are there any empty bins in images? Our assumption is that Monet images might not make use of the full range of intensity values (based on the histograms)

In [ ]:
zero_values_monet = {c: [np.count_nonzero(hist[i].ravel()==0) for hist in monet_histograms] for i,c in enumerate(color)}
zero_values_photo = {c: [np.count_nonzero(hist[i].ravel()==0) for hist in photo_histograms] for i,c in enumerate(color)}

In [ ]:
boxplots_for_comparison(zero_values_monet,zero_values_photo, "Number of not used intensity values Monet and Photo" )